In [2]:
#%matplotlib nbagg
#%matplotlib notebook
import matplotlib.pyplot as plt 
%matplotlib qt
import numpy as np
import pyxem as pxm
import hyperspy.api as hs
from pyxem.libraries.calibration_library import CalibrationDataLibrary
from pyxem.generators.calibration_generator import CalibrationGenerator
#%matplotlib inline

In [3]:
s1=hs.load('STEM SI.dm4')

In [4]:
s=s1[2]

In [108]:
#search for sub-folder name
import os
import re
def sub_folder_number():
    # Get the current working directory
    current_directory = os.getcwd()

    # Extract the folder name from the path
    folder_name = os.path.basename(current_directory)

    # Use regular expression to extract the number from the folder name
    number = re.search(r'\((\d+)\)', folder_name)

    if number:
        extracted_number = number.group(1)
        print(extracted_number)  # Output:  the number inside the parentheses in your current folder name
        return int(extracted_number)
    else:
        print("Number not found in the folder name.")
sub_folder=sub_folder_number()

3


In [5]:
s.set_signal_type('electron_diffraction')

In [11]:
s.plot() #please move to the ROI in the navigation image

In [12]:
roi = hs.roi.CircleROI(cx=-2.6,cy=-1.13, r_inner=0, r=0.5) # mask on specific reflection point

In [13]:
roi.gui() # after run this code, drag the ROI to the flection point

In [127]:
s.plot_integrated_intensity(roi)

In [151]:
SI_image_coord=(252,6) # your input  # (x,y):the coordinations of picked spot in the SI image. Read from Diffraction SI Signal
x=roi[0]
y=roi[1]  

Center_of_reflection=(x,y) 
Center_of_reflection_in_pixel= (Center_of_reflection[0]/s.axes_manager[3].scale,Center_of_reflection[1]/s.axes_manager[3].scale) # # (x,y):the coordinations of the picked reflection point in the Diffraction pattern, copied from roi.gui()
DP_image=s.inav[SI_image_coord] #retrieve DP data
DP_image_data=DP_image.data
radius=0.5 # your input 
radius_in_pixel=radius/s.axes_manager[3].scale

In [152]:
roi2 = hs.roi.CircleROI(cx=Center_of_reflection[0],cy=Center_of_reflection[1], r_inner=0, r=radius) # this is the same as roi, but for selecting VDF image

In [153]:
vdf = s.get_integrated_intensity(roi2)
vdf.plot()
vdf_data=vdf.data.data
vdf_pixel_to_nm=s.axes_manager[1].scale
DP_pixel_to_nm=s.axes_manager[3].scale

In [185]:
import os

# Check if the file containing image_num exists
if os.path.exists("image_num.txt"):
    # If it exists, read the value from the file
    with open("image_num.txt", "r") as file:
        image_num = int(file.read())
else:
    # If it doesn't exist, initialize image_num to 0
    image_num = 0

# Increment image_num
image_num += 1

# Write the updated value back to the file
with open("image_num.txt", "w") as file:
    file.write(str(image_num))

# Print the updated value of image_num
print(image_num)

1


In [157]:
# Check if the dictionaries do not exist in the global scope and create them if necessary
if 'manual_measured_size' not in globals():
    manual_measured_size = {}

if 'semi_manual_measured_size' not in globals():
    semi_manual_measured_size = {}
    
if 'coord_in_SI' not in locals() or 'dict_for_image_storage' not in globals():
    coord_in_SI=[]
coord_in_SI.append(SI_image_coord) # Store the picked spot from SI image.

# Check if dict_for_image_storage exists, if not, initialize it as an empty dictionary
if 'dict_for_image_storage' not in locals() or 'dict_for_image_storage' not in globals():
    dict_for_image_storage = {}
VDF_file_name = '1-{}-VDF'.format(sub_folder)
DP_file_name = '1-{}-DP'.format(sub_folder)
    
# Check if '1-2-VDF' and '1-2-DP' keys exist in dict_for_image_storage
if VDF_file_name not in dict_for_image_storage:
    dict_for_image_storage[VDF_file_name] = {}
if DP_file_name not in dict_for_image_storage:
    dict_for_image_storage[DP_file_name] = {}

# Assuming vdf_data, DP_image_data, SI_image_coord, Center_of_reflection_in_pixel, and radius_in_pixel are defined elsewhere
# Update data in the dictionary
dict_for_image_storage[VDF_file_name][image_num] = (vdf_data, vdf_pixel_to_nm)  # (image#, vdf data)
dict_for_image_storage[DP_file_name][image_num] = (DP_image_data, DP_pixel_to_nm, SI_image_coord, Center_of_reflection_in_pixel, radius_in_pixel)

# Save images in text files
np.savetxt('1-{}-{}-VDF.txt'.format(sub_folder,image_num), dict_for_image_storage[VDF_file_name][image_num][0])  # Save VDF data in a text file
np.savetxt('1-{}-{}-DP.txt'.format(sub_folder,image_num), dict_for_image_storage[DP_file_name][image_num][0])  # Save DP image data in a text file                        


In [159]:
#Measure the linewidth mannually
from matplotlib.widgets import SpanSelector
if 'manual_coord' not in globals():
    manual_coord= {}

# Assuming dict_for_image_storage, image_num, vdf_pixel_to_nm, and vdf_data are defined
vdf_data = dict_for_image_storage[VDF_file_name ][image_num][0]
vdf_scale_factor = vdf_pixel_to_nm  # Adjust the scale based on vdf_pixel_to_nm

# Lists to store line coordinates and measured lengths
line_coords = []
line_lengths = []

# Function to be called when a line is selected
def onselect(xmin, xmax):
    x_coords = [xmin, xmax]
    line_coords.append(x_coords)
    length_nm = (xmax - xmin)
    line_lengths.append(length_nm)
    print(f'Measured Length: {length_nm:.2f} nm')
    plt.axhline(y=0, color='red', linewidth=2)  # Draw a red line after each measurement
    plt.draw()

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(vdf_data, cmap='gray', extent=[0, vdf_data.shape[1] * vdf_scale_factor, 0, vdf_data.shape[0] * vdf_scale_factor])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (nm)')  # Y-axis label indicating nanometers

# SpanSelector to draw lines interactively
span = SpanSelector(ax, onselect, 'horizontal', useblit=True, rectprops=dict(alpha=0.5, facecolor='red'))
color_list=['red','green','blue','yellow','white']
plt.show()

# Print the final adjusted lengths
print('Final Adjusted Lengths:')
for i, length in enumerate(line_lengths):
    print(f'Line {i + 1}: {length:.2f} nm')

Final Adjusted Lengths:
Measured Length: 31.49 nm


In [160]:
#run the following script to save the figure with measurements and the results
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(vdf_data, cmap='gray', extent=[0, vdf_data.shape[1] * vdf_scale_factor, 0, vdf_data.shape[0] * vdf_scale_factor])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (nm)')
color_list=['red','green','blue','yellow','white']
plot_position=[-30,vdf_data.shape[0]* vdf_scale_factor+20]
manual_coord[image_num]=np.array([line_coords])
for i in range(len(line_coords)):
    plt.axvline(x=line_coords[i][0], color=color_list[i], linestyle='--', linewidth=2)
    plt.axvline(x=line_coords[i][1], color=color_list[i], linestyle='--', linewidth=2)
    particle_num=i+1
    plt.text(line_coords[i][0], plot_position[i%2],'particle %s \n %s nm'%(particle_num,round(line_lengths[i],1)), fontsize=10, ha='center', va='center',c=color_list[i])
plt.show()
plt.savefig('1-{}-{}-VDF-{}nm_manualy.tif'.format(sub_folder,image_num, line_lengths))
line_lengths_array = np.array(line_lengths)
line_lengths_round=np.round(line_lengths_array,1)
np.savetxt('1-{}-{}-VDF-{}nm-mannauly.txt'.format(sub_folder,image_num, line_lengths_round),line_lengths_round)
mannual_measured_size[image_num]=line_lengths

In [161]:
print('mannual_measured_size:',mannual_measured_size,'\nmannual_coordination:',manual_coord)

mannual_measured_size: {1: [37.52170876689976], 2: [17.58010131036562], 3: [17.58010131036562, 19.154438741144645], 4: [21.515944887313168, 11.807530730842586, 12.594699446232084], 5: [18.892049169348127, 12.069920302639105, 14.169036877011138], 6: [10.495582871860051], 7: [28.33807375402222], 8: [18.892049169348184], 9: [11.020362015453088], 10: [31.486748615580154]} 
mannual_coordination: {1: array([[[ 7.34690801, 44.86861678]]]), 2: array([[[54.05225179, 71.6323531 ]]]), 3: array([[[ 97.34653114, 114.92663245],
        [114.92663245, 134.08107119]]]), 4: array([[[138.01691476, 159.53285965],
        [161.10719708, 172.91472781],
        [176.85057139, 189.44527084]]]), 5: array([[[198.62890585, 217.52095502],
        [218.30812373, 230.37804404],
        [230.64043361, 244.80947049]]]), 6: array([[[309.09491558, 319.59049845]]]), 7: array([[[338.22015805, 366.5582318 ]]]), 8: array([[[401.45604485, 420.34809402]]]), 9: array([[[473.35078752, 484.37114954]]]), 10: array([[[479.910526

In [162]:
#Semi-manual


# Define Region of Interest (ROI)
x_prime=7.5 #your input
y_prime=17. #your input
import numpy as np
import matplotlib.pyplot as plt
if 'semi_manual_coord' not in globals():
    semi_manual_coord= {}

vdf_data_with_scale = dict_for_image_storage[VDF_file_name][image_num][0]
vdf_scale_factor = vdf_pixel_to_nm  # Adjust the scale based on vdf_pixel_to_nm
plt.figure(figsize=(6, 6))
plt.imshow(vdf_data_with_scale, cmap='gray', extent=[0, vdf_data_with_scale.shape[1] * vdf_scale_factor, 0, vdf_data_with_scale.shape[0] * vdf_scale_factor])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (nm)')  # X-axis label indicating nanometers
plt.ylabel('Y (nm)')  # Y-axis label indicating nanometers


# Define Region of Interest (ROI)
x_prime=6 #your input
y_prime=17. #your input

number_start=round(x_prime/vdf_scale_factor)
number_end=round(y_prime/vdf_scale_factor)

threshold = 40000  # Threshold value for the signal


roi_data = vdf_data[number_start:number_end,:]  # Extract ROI

mid= round((y_prime-x_prime)/2/vdf_scale_factor)
start_col=[]
end_col = []
start_Found=False

for col in range(roi_data.shape[1]):
    if roi_data[mid,col]>threshold and start_Found is False:
        start_col.append(col)
        start_Found=True
    if roi_data[mid,col]<threshold and start_Found:
        end_col.append(col)
        start_Found=False
if len(start_col)==(len(end_col)+1): #if start_col is larger than end_col
    start_col.pop() #remove the last added one

combined_cols = list(zip(start_col, end_col))

combined_cols_array = np.array(combined_cols)
semi_manual_coord[image_num]=combined_cols_array

# Plot the VDF data
plt.imshow(vdf_data, cmap='gray', extent=[0, vdf_data.shape[1], 0, vdf_data.shape[0]])
plt.title('VDF Data (Image {})'.format(image_num))
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')

# Draw the line on the plot
color_list=['red','green','blue','yellow','white']
plot_position=[-30,vdf_data.shape[0]* vdf_scale_factor+20]
for i in range(len(start_col)):
    if start_col is not None and end_col is not None:
        plt.axvline(x=start_col[i], color=color_list[i], linestyle='--', linewidth=2)
        plt.axvline(x=end_col[i], color=color_list[i], linestyle='--', linewidth=2)

plt.show()
start_col_array=np.array(start_col)
end_col_array=np.array(end_col)
measured_length = (end_col_array - start_col_array) * s.axes_manager[1].scale
print('Measured Length:', measured_length, 'nm')

# Save the plot
plt.savefig('1-{}-{}-VDF-{}nm_semi_mannualy.tif'.format(sub_folder,image_num, np.round(measured_length,1)))
np.savetxt('1-{}-{}-VDF-{}nm-semi_mannauly.txt'.format(sub_folder,image_num, np.round(measured_length,1)),measured_length)
semi_mannual_measured_size[image_num]=measured_length


Measured Length: [33.77815068] nm


In [163]:
print('semi_mannual_measured_size:',semi_mannual_measured_size,'\nsemi_mannual_coordination:',semi_manual_coord)

semi_mannual_measured_size: {1: array([33.77815068]), 2: array([13.90865028]), 3: array([11.92170024, 17.88255036]), 4: array([17.88255036,  9.9347502 ]), 5: array([15.89560032, 13.90865028]), 6: array([9.9347502]), 7: array([31.79120064]), 8: array([21.85645044]), 9: array([9.9347502]), 10: array([33.77815068])} 
semi_mannual_coordination: {1: array([[ 4, 21]]), 2: array([[28, 35]]), 3: array([[51, 57],
       [58, 67]]), 4: array([[71, 80],
       [82, 87]]), 5: array([[101, 109],
       [111, 118]]), 6: array([[156, 161]]), 7: array([[169, 185]]), 8: array([[201, 212]]), 9: array([[237, 242]]), 10: array([[241, 258]])}


In [180]:
#export the measured sizes from mannual or semi-mannual
list_semi_mannual_measured_size=[]
for i in range(len(semi_mannual_measured_size)):
    list_semi_mannual_measured_size.append(semi_mannual_measured_size[i+1])

def flatten(data):
    flat_list = []
    for item in data:
        if isinstance(item, (int, float)):
            flat_list.append(item)
        elif isinstance(item, list):
            flat_list.extend(flatten(item))
        elif isinstance(item, np.ndarray):
            flat_list.extend(item.tolist())
    return flat_list

flat_list_semi_mannual_measured_size = flatten(list_semi_mannual_measured_size)

np.savetxt('Total semi_mannual_measured_size.txt',flat_list_semi_mannual_measured_size )

list_mannual_measured_size=[]

for i in range(len(mannual_measured_size)):
    list_mannual_measured_size.append(mannual_measured_size[i+1])


flat_list_mannual_measured_size = flatten(list_mannual_measured_size)
np.savetxt('Total mannual_measured_size.txt',flat_list_mannual_measured_size )


In [165]:
#Save all the files # dicts
import pickle


dicts_to_save = {
    'mannual_measured_size': mannual_measured_size,
    'semi_mannual_measured_size': semi_mannual_measured_size,
    'dict_for_image_storage': dict_for_image_storage,
    'semi_manual_coord':semi_manual_coord,
    'manual_coord': manual_coord
}

with open('saved_dicts.pkl', 'wb') as file:
    pickle.dump(dicts_to_save, file)

In [280]:
# Load the dictionary containing our data
with open('saved_dicts.pkl', 'rb') as file:
    loaded_dicts = pickle.load(file)

# Access individual dictionaries using their variable names
#mannual_measured_size = loaded_dicts['mannual_measured_size']
#semi_mannual_measured_size = loaded_dicts['semi_mannual_measured_size']
#dict_for_image_storage = loaded_dicts['dict_for_image_storage']


In [166]:
#After all the measurements done. Plotting all the DF images
num_images = len(dict_for_image_storage[VDF_file_name])
fig, axes = plt.subplots(num_images, 1, figsize=(8, 6*num_images))
semi_mannual_measured_size_round = {k: round(v, 1) if not isinstance(v, np.ndarray) else np.round(v, 1) for k, v in semi_mannual_measured_size.items()}
mannual_measured_size_round = {k: [round(num, 1) for num in v] for k, v in mannual_measured_size.items()}
# Iterate through the dictionary and plot images vertically
for i in range(1, num_images + 1):
    data, scale_factor = dict_for_image_storage[VDF_file_name][i]
    ax = axes[i - 1]  # Select the appropriate subplot
    ax.imshow(data, cmap='gray', extent=[0, data.shape[1] * scale_factor, 0, data.shape[0] * scale_factor])
    ax.set_title('VDF (image{} from point {} with sizes {} and {} measured by semi- or mannually)'.format(i, dict_for_image_storage['1-3-DP'][i][2], mannual_measured_size_round[i], semi_mannual_measured_size_round[i]))
    ax.set_xlabel('X (nm)')  # Adjust the label according to your data unit
    ax.set_ylabel('Y (nm)')  # Adjust the label according to your data unit

plt.tight_layout()  # Adjust spacing between subplots for better visualization
plt.savefig('Size measuremnts.tif') 
plt.show()

In [179]:
# Calculate the number of rows and columns for subplots
num_images = len(dict_for_image_storage[DP_file_name])
num_cols = int(np.sqrt(num_images))  # Number of columns (square root of num_images)
num_rows = num_images // num_cols  # Number of rows
if num_images % num_cols != 0:
    num_rows += 1
# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6*num_rows))

# Iterate through the dictionary and plot images horizontally
for i in range(num_images):
    row = i // num_cols  # Calculate the row index
    col = i % num_cols  # Calculate the column index
    
    data = dict_for_image_storage[DP_file_name][i + 1][0]
    scale_factor = dict_for_image_storage[DP_file_name][i + 1][1]
    ax = axes[row, col]  # Select the appropriate subplot
    max_intensity = data.max()  # Calculate the maximum intensity value of the image
    upper_intensity_limit = max_intensity / 10  # Set the upper intensity limit to 1/3 of the maximum intensity

    # Plot the image with adjusted upper intensity limit
    ax.imshow(data, cmap='gray', extent=[0, data.shape[1] * scale_factor, 0, data.shape[0] * scale_factor], vmax=upper_intensity_limit)
    ax.set_title('DP (image {} from point {} with reflection point at {})'.format(i + 1, dict_for_image_storage[DP_file_name][i + 1][2], 
                                                                                  dict_for_image_storage[DP_file_name][i + 1][3]))
    ax.set_xlabel('X (nm)')  # Adjust the label according to your data unit
    ax.set_ylabel('Y (nm)')  # Adjust the label according to your data unit
    circle_points = 100
    radius_in_nm = dict_for_image_storage['1-3-DP'][i + 1][4]*scale_factor  # Radius in pixel
    
    center_x, center_y = [coord * scale_factor for coord in dict_for_image_storage[DP_file_name][i + 1][3]]
    center_x += image_width / 2
    center_y = image_width / 2 - center_y
    
    for angle in range(0, 360, 360 // circle_points):
        x = center_x - radius_in_nm * cos(deg2rad(angle))
        y = center_y + radius_in_nm * sin(deg2rad(angle))
        ax.plot(x, y, marker='o', markersize=1, color='blue')

plt.tight_layout()  # Adjust spacing between subplots for better visualization
plt.savefig('All_DPs.tif')  # Save the plot as an image file
plt.show()

In [175]:
len(dict_for_image_storage[DP_file_name])

10